In [1]:
from PIL import Image
import os
import pathlib
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle as sk_shuffle
import time
import os

import matplotlib.pyplot as plt

from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler

import glob2
import pdb
import ipdb

In [2]:

SEED = 42


labels_dict = {'1':1, '2':2, '3':3, '4':4, '5':5,
               '6':6, '7':7, '8':8, '9':9, '10':10,
               '11':11, '12':12, '13':13, '14':14, '15':15,
               '16':16, '17':17, '18':18, '19':19, '20':20,
               '21':21, '22':22, '23':23, '24':24, '25':25,
               '26':26, '27':27, '28':28, '29':29, '30':30,
               '31':31, '32':32, 'a':33, 'b':34, 'c':35, 'd':36,
               'e':37, 'f':38, 'g':39, 'h':40, 'i':41, 'j':42, 'k':43,
               'l':44, 'm':45, 'n':46, 'o':47, 'p':48, 'q':49, 'r':50, 's':51, 't':52,
               'conecut':53, 'caries':54, 'caries2':54,'caries3':54,'crown':55, 'bridge':56,
               'gutta-percha':57, 'pin':58, 'post':59, 'buildup':60, 'calculus':61, 'sealer':62,
               'cement':63, 'bracket':64, 'retainer':65, 'instrument':66, 'implant':67, 'screw':68,
               'veneer':69, 'silver-point':70, 'space-maintainer':71, 'core':72, 'base':73,
               'temporary-filling':74, 'medicament':75, 'framework':76, 'liner':77, 'onlay-composite':78,
               'onlay-metal':79, 'onlay-ceramic':80, 'inlay-ceramic':81, 'inlay-composite':82, 'inlay-metal':83,
               'filling-composite':84, 'filling-glass':85, 'filling-metal':86}

reverse_labels_dict = {v: k for k, v in labels_dict.items()}

labels_dict_no_baby = {'1':1, '2':2, '3':3, '4':4, '5':5,
               '6':6, '7':7, '8':8, '9':9, '10':10,
               '11':11, '12':12, '13':13, '14':14, '15':15,
               '16':16, '17':17, '18':18, '19':19, '20':20,
               '21':21, '22':22, '23':23, '24':24, '25':25,
               '26':26, '27':27, '28':28, '29':29, '30':30,
               '31':31, '32':32, 'conecut':33, 'caries':34, 'caries2':34,'caries3':34}
regions_dict_all = {'Anterior1MandibleB':0,
               'Anterior2MaxillaL':1,
               'Anterior2MaxillaR':2,
               'Anterior3MandibleL':3,
               'Anterior3MandibleR':4,
               'Anterior3MaxillaL':5,
               'Anterior3MaxillaR':6,
               'Molar3JawregionL':7,
               'Molar3JawregionR':8,
               'Molar3MandibleL':9,
               'Molar3MandibleR':10,
               'Molar3MaxillaL':11,
               'Molar3MaxillaR':12,
               'Anterior1MandibleL':13,
               'Premolar2JawregionR':14,
               'Premolar2MandibleR':15,
               'Premolar2MaxillaL':16,
               'Premolar2MaxillaR':17,
               'Anterior1MaxillaB':18,
               'Premolar2MandibleL':19,
               'Premolar2JawregionL':20,
               'Anterior1MaxillaR':21,
               'Anterior1MaxillaL':22,
               'Anterior2MaxillaB':23,
               'Anterior2MandibleL':24,
               'Anterior1MandibleR':25}

regions_dict_standard = {'Anterior2MaxillaR':0,
                         'Anterior3MaxillaR':1,
                         'Anterior1MandibleB':2,
                         'Molar3MandibleR':3,
                         'Molar3MaxillaL':4,
                         'Anterior2MaxillaL':5,
                         'Anterior3MaxillaL':6,
                         'Molar3MaxillaR':7,
                         'Anterior3MandibleL':8,
                         'Premolar2MaxillaL':9,
                         'Premolar2MandibleR':10,
                         'Premolar2MandibleL':11,
                         'Molar3JawregionL':12,
                         'Anterior3MandibleR':13,
                         'Molar3JawregionR':14,
                         'Premolar2JawregionL':15,
                         'Premolar2JawregionR':16,
                         'Premolar2MaxillaR':17,
                         'Anterior1MaxillaB':18,
                         'Molar3MandibleL':19}


tooth_dict = {'1':1, '2':2, '3':3, '4':4, '5':5,
               '6':6, '7':7, '8':8, '9':9, '10':10,
               '11':11, '12':12, '13':13, '14':14, '15':15,
               '16':16, '17':17, '18':18, '19':19, '20':20,
               '21':21, '22':22, '23':23, '24':24, '25':25,
               '26':26, '27':27, '28':28, '29':29, '30':30,
               '31':31, '32':32, 'a':33, 'b':34, 'c':35, 'd':36,
               'e':37, 'f':38, 'g':39, 'h':40, 'i':41, 'j':42, 'k':43,
               'l':44, 'm':45, 'n':46, 'o':47, 'p':48, 'q':49, 'r':50, 's':51, 't':52}
prior_dict = {'crown':1, 'bridge':2,
               'gutta-percha':3, 'pin':4, 'post':5, 'buildup':6, 'calculus':7, 'sealer':8,
               'cement':9, 'bracket':20, 'retainer':11, 'instrument':12, 'implant':13, 'screw':14,
               'veneer':15, 'silver-point':16, 'space-maintainer':17, 'core':18, 'base':19,
               'temporary-filling':20, 'medicament':21, 'framework':22, 'liner':23, 'onlay-composite':24,
               'onlay-metal':25, 'onlay-ceramic':26, 'inlay-ceramic':27, 'inlay-composite':28, 'inlay-metal':29,
               'filling-composite':30, 'filling-glass':31, 'filling-metal':32}

filling_dict = {'temporary-filling':1, 'filling-composite':2, 'filling-glass':3, 'filling-metal':4}
reverse_filling_dict = {v: k for k, v in filling_dict.items()}


tooth_list = list(tooth_dict.keys())

In [20]:


class read_data:
    def __init__(self, path, issynthetic=False, imrange=[0,55000]):
        path = os.path.join(path, 'images')
        self.image_path = pathlib.Path(path)
        self.issynthetic = issynthetic
        self.imrange = imrange
        
    def create_dataset(self):
        img_paths = []
        if not self.issynthetic:
            # use only standard views
            for std_view in regions_dict_standard:
                pathk = '*/' + std_view + '/im.jpg'
                img_paths += list(self.image_path.glob(pathk))
        else:
            # use only standard views
            for std_view in regions_dict_standard:
                pathk = '*/' + std_view + '/*/im.jpg'
                img_paths += list(self.image_path.glob(pathk))
        
        in_impaths = img_paths if self.imrange=='all' else img_paths[self.imrange[0]:self.imrange[1]]
        
        return in_impaths

class segregate:
    def __init__(self, all_impaths, segregate_label):
        self.all_ims =  all_impaths
        self.sg_label = segregate_label
        self.sg_filter = lambda x: self.sg_label in x and x.endswith('.jpg')
        
    def segregate_data(self):
        
        noCaries_im = []
        noCaries_lbl = []
        caries1_im = []
        caries1_lbl = []
        caries2_im = []
        caries2_lbl = []
        
        for impath in self.all_ims:
            impath = impath.as_posix()
            lbpath = os.path.dirname(impath).replace('images', 'masks', 1)
            lbpath_dict = {self.sg_label:[]}
            sg_lbpath = [os.path.join(lbpath, lbname) for lbname in os.listdir(lbpath) if self.sg_filter(lbname)]
            if len(sg_lbpath) > 0:
                if len(sg_lbpath)==1:
                    lbpath_dict[self.sg_label].extend(sg_lbpath)
                    caries1_im.append(impath)
                    caries1_lbl.append(lbpath_dict)
                else:
                    lbpath_dict[self.sg_label].extend(sg_lbpath)
                    caries2_im.append(impath)
                    caries2_lbl.append(lbpath_dict)
            else:
                noCaries_im.append(impath)
                noCaries_lbl.append(lbpath_dict)
        
        return noCaries_im, noCaries_lbl, caries1_im, caries1_lbl, caries2_im, caries2_lbl

        
class CariesExtBase:
    def __init__(self, path, segregate_label,  oversample_factor=None, im_range = [0, 55000] ,issynthetic=False, test_size=0.2, isTrain=True):
                 
        
        self.isTrain = isTrain
        
        self.data_load = read_data(path, issynthetic, im_range)
        all_impaths = self.data_load.create_dataset()
        print('Dataset Loaded = {}'.format(len(all_impaths)))
        
        self.segregate = segregate(all_impaths, segregate_label)
        nocaries_im, nocaries_lbl, caries1_im, caries1_lbl, caries2_im, caries2_lbl = self.segregate.segregate_data()
        print('Total Data with no caries present: {}'.format(len(nocaries_im)))
        print('Total Data with 1 caries present: {}'.format(len(caries1_im)))
        print('Total Data with 2 caries present: {}'.format(len(caries2_im)))
        
        nocaries_trainims, nocaries_valims, nocaries_trainlbls, nocaries_vallbls = self.split_data(nocaries_im, nocaries_lbl, test_size)
        caries1_trainims, caries1_valims, caries1_trainlbls, caries1_vallbls = self.split_data(caries1_im, caries1_lbl, test_size)
        caries2_trainims, caries2_valims, caries2_trainlbls, caries2_vallbls = self.split_data(caries2_im, caries2_lbl, test_size)
        
        valims_caries = caries2_valims + nocaries_valims
        vallbls_caries = caries2_vallbls + nocaries_vallbls
        self.val_impaths, self.val_lbpaths = sk_shuffle(valims_caries, vallbls_caries, random_state=42)
        print('Val data size = {}'.format(len(self.val_impaths)))
        
        caries2_ovrims, caries2_ovrlbls = self.oversample(caries2_trainims, caries2_trainlbls, oversample_factor)
        print('Total Data with 2 caries present (After oversampling): {}'.format(len(caries2_ovrims)+len(caries2_valims)))
#         nocaries_ovrims, nocaries_ovrlbls = self.oversample(nocaries_trainims, nocaries_trainlbls, oversample_factor[1])
        
        trainims_caries = caries2_ovrims + nocaries_trainims
        trainlbls_caries = caries2_ovrlbls  + nocaries_trainlbls
        self.train_impaths, self.train_lbpaths = sk_shuffle(trainims_caries, trainlbls_caries, random_state=42)
        print('Train data size after oversampling = {}'.format(len(self.train_impaths)))
        
    
    def split_data(self, impaths, lbpaths, test_size):
            
        train_ims, val_ims, train_lbls, val_lbs = train_test_split(impaths, lbpaths, test_size= test_size, random_state=42)
        return train_ims, val_ims, train_lbls, val_lbs
        
    
    def oversample(self, impaths, lbpaths, oversample_factor=None):
        
        over_imgs, over_lbls = [], []
        if isinstance(oversample_factor, int) and oversample_factor >0:
            for impath, lbpath_dict in zip(impaths, lbpaths):
                over_lbls += [lbpath_dict]*oversample_factor
                over_imgs += [impath]*oversample_factor
            over_imgs, over_lbls = sk_shuffle(over_imgs, over_lbls, random_state=42)
            return over_imgs, over_lbls
        else:
            return impaths, lbpaths
    
    def __len__(self):
        return len(self.train_impaths) + len(self.val_impaths)
    
    def __getitem__(self, idx):
        if self.isTrain:
            return self.train_impaths[idx], self.train_lbpaths[idx]
        else:
            return self.val_impaths[idx], self.val_lbpaths[idx]
    
mark1=time.time()
filling_test = CariesExtBase('/home/rohan/Datasets/prior_clean/train', segregate_label='caries', oversample_factor=2, 
                            issynthetic=False, im_range = 'all', test_size=0.1, isTrain=True)
print('dataset time = {} secs'.format(time.time()-mark1)) 


Dataset Loaded = 60178
Total Data with no caries present: 27426
Total Data with 1 caries present: 15624
Total Data with 2 caries present: 17128
Val data size = 4456
Total Data with 2 caries present (After oversampling): 32543
Train data size after oversampling = 55513
dataset time = 4.721735000610352 secs


('/home/rohan/Datasets/prior_clean/train/images/1.190926791160849.95980433801285866127682..042672.0671514770.7/Premolar2JawregionR/im.jpg',
 {'caries': ['/home/rohan/Datasets/prior_clean/train/masks/1.190926791160849.95980433801285866127682..042672.0671514770.7/Premolar2JawregionR/caries3.jpg',
   '/home/rohan/Datasets/prior_clean/train/masks/1.190926791160849.95980433801285866127682..042672.0671514770.7/Premolar2JawregionR/caries2.jpg',
   '/home/rohan/Datasets/prior_clean/train/masks/1.190926791160849.95980433801285866127682..042672.0671514770.7/Premolar2JawregionR/caries.jpg']})